# <font color=blue> steps</font>
0. get shapefile for city council districts
1. create chloropleth for city council districts
2. begin to restyle the drop downs 
3. fix size read error with mapbox not rendering
3. create a chart Class  and make the bar charts instances of the main chart class 

In [ ]:

import warnings
warnings.filterwarnings(action='ignore')

from datetime import datetime

import charts
import data
from data import dframe
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)

In [ ]:
## update layout ##
#from IPython.display import Image
from IPython import display
display.Image(filename='IMG_3814.JPG')

In [ ]:
dframe['fiscal_year_requested'].unique()

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import jupyterlab_dash


In [ ]:
# jupyterlab -dash extension for viewing dash apps
viewer = jupyterlab_dash.AppViewer()

In [1]:
app = dash.Dash('update-layout')
app.css.append_css({'external_url': 'https://cdn.rawgit.com/plotly/dash-app-stylesheets/2d266c578d2a6e8850ebce48fdb52759b2aef506/stylesheet-oil-and-gas.css'})  


app.layout = html.Div([
    # dropdown widget above row 1 
    html.Div([
        html.Div([dcc.Dropdown(
        id = 'input-for-first-4-charts',
        options = [{'label':year, 'value': year} for year in dframe['fiscal_year_requested'].unique()],
        multi = False,
        value = 2015)],  
                 
        className = 'two columns'
                )],
        
    className = 'twelve columns'),
    
    # row 1
    html.Div([
        html.Div(
        [dcc.Graph(
            id = 'volume-scatterchart',
            config = {} # for configuration of plotly charts,  
        )],
            
        className = 'four columns',
        style = {'margin-top': '20'}),
    
    html.Div(
        [dcc.Graph(
            id = 'scattermap',
            config = {} # for configuration of plotly charts,  
        )], 
        
        className = 'four columns',
        style = {'margin-top': '20'}),
    
    html.Div(
        [dcc.Graph(
            id = 'choropleth-map',
            config = {} # for configuration of plotly charts,  
        )], 
        
        className = 'four columns',
        style = {'margin-top': '20'})
            ]),
    
    # dropdown widget above row 2 
    html.Div([
        html.Div([dcc.Dropdown(
        id = 'input-for-heatmaps',
        options = [{'label':option, 'value': option} for option in ['count','percentage']],
        multi = False,
        value = 'count')],  
                 
        className = 'two columns'
                )],
        
    className = 'twelve columns'),
    
    # row 2
    html.Div([
        html.Div(
        [dcc.Graph(
            id = 'heatmap-weekday',
            config = {} # for configuration of plotly charts,  
        )], 
            
        className = 'four columns',
        style = {'margin-top': '20'}),
    
    html.Div(
        [dcc.Graph(
            id = 'heatmap-month',
            config = {} # for configuration of plotly charts,  
        )], 
        
        className = 'four columns',
        style = {'margin-top': '20'}),
    
    html.Div(
        [dcc.Graph(
            id = 'donut-chart',
            config = {} # for configuration of plotly charts,  
        )], 
        
        className = 'four columns',
        style = {'margin-top': '20'})
            ]),

# TODO: change function parameters for open_vs_completed from single letter
# frequencies like 'M', 'D' to more readable strings like 'monthly','daily' for 
# the dropdown input. 
    
    # dropdown widget above row 3 
    html.Div([
        html.Div([dcc.Dropdown(
        id = 'input-for-performance-charts',
        options = [{'label':period, 'value': period} for period in ['D','W','M','Q','A']],
        multi = False,
        value = 'Q')],  
                 
        className = 'two columns'
                )],
        
    className = 'twelve columns'),
    
    # row 3
    html.Div([
        html.Div(
        [dcc.Graph(
            id = 'backlog-linechart',
            config = {} # for configuration of plotly charts,  
        )], 
            
        className = 'four columns',
        style = {'margin-top': '20'}),
    
    html.Div(
        [dcc.Graph(
            id = 'ontime-kpi',
            figure = charts.pct_closed_ontime(dframe),
            config = {} # for configuration of plotly charts,  
        )], 
        
        className = 'four columns',
        style = {'margin-top': '20'}),
    
    html.Div(
        [dcc.Graph(
            id = 'pm2cm-kpi',
            figure = charts.pm2cm_kpi(dframe=dframe,
                                     pm_types=data.preventative_types,
                                     cm_types=data.corrective_types),
            config = {} # for configuration of plotly charts,  
        )], 
        
        className = 'four columns',
        style = {'margin-top': '20'})
            ]),
    
])

# TODO: abstract fiscalyear column as a default value in duration_vs_volume so 
# it is not necessary to pass the string to the callback function

## Callbacks for dashboard interactivity

@app.callback(
    dash.dependencies.Output(
        component_id = 'volume-scatterchart',
        component_property = 'figure'),
    
    [dash.dependencies.Input(
        component_id = 'input-for-first-4-charts',
        component_property = 'value')])
def generate_scatterplot(dropdown_year):
    return charts.duration_vs_volume(dframe=dframe, fiscalyear_column='fiscal_year_requested',year=dropdown_year)

# TODO: add callbacks for scattermap & choropleth map

@app.callback(
    dash.dependencies.Output(
        component_id = 'heatmap-weekday',
        component_property = 'figure'),
    
    [dash.dependencies.Input(
        component_id = 'input-for-heatmaps',
        component_property = 'value')])
def generate_weekday_heatmap(dropdown_zvalue):
    return charts.weekday_completion(dframe=dframe, z_values=dropdown_zvalue)
    
@app.callback(
    dash.dependencies.Output(
        component_id = 'heatmap-month',
        component_property = 'figure'),
    
    [dash.dependencies.Input(
        component_id = 'input-for-heatmaps',
        component_property = 'value')])
def generate_weekday_heatmap(dropdown_zvalue):
    return charts.month_completion(dframe=dframe, z_values=dropdown_zvalue)
    

@app.callback(
    dash.dependencies.Output(
        component_id = 'backlog-linechart',
        component_property = 'figure'),
    
    [dash.dependencies.Input(
        component_id = 'input-for-performance-charts',
        component_property = 'value')])
def generate_backlog_chart(dropdown_frequency):
    return charts.open_vs_completed(dframe=dframe, frequency=dropdown_frequency)


## display app in jupyter notebook
viewer.show(app)

NameError: name 'dash' is not defined

## <font color=red> look at what happens or add in as parameter, for when pm to cm is calculated with only completed pms (but still open cms)</font>